In [0]:
# import pyspark
from delta import *

In [0]:
# builder = pyspark.sql.SparkSession.builder.appName("MetadataDeltalakeETL") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

In [0]:
# spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [0]:
# spark

In [0]:
# Page Break

# Load Metadata and Save as Deltatable

* NOTE:
    * This `meatadata.tsv` is total make up mock for demonstration.
    * To simplify thing, it is prepared in denormalized flat table form.

In [0]:
# meta_src = "./data/metadata.csv"

meta_src = "dbfs:/Workspace/Repos/danny.wong@databricks.com/genomic-bigdata-spark/data/metadata.csv" #not working

In [0]:
meta_df = spark.read.csv(meta_src, header=True)

In [0]:
meta_df.printSchema()

In [0]:
# Page Break

# Data Warehouse Models

> Consider: some ETL or complex transformation have done here!! :)

* At this point, we can take one step back and think about leveraging Data Warehouse data models.
 * Some examples but not limited to:
    * Multi-Dimensional data model such as Star schema, Snowflake schema
        * For example, wrap around our `somatic_table` as Fact table and, build metadata as surrounding dimension table(s)
    * Data Vault data modelling such as arranging in Hub, Satellite, Links concepts (depends on data stages i.e. Bronze, Silver, Gold, Platinum)
    * Or, simple flat table, Data Mart or, even some simplified Relation model if that suit for the use case

In [0]:
# Page Break

## QA TRANSFORMED DATAFRAME

In [0]:
meta_df.select("SampleID").distinct().show()

In [0]:
meta_df \
    .select("SequenceRunName", "Timestamp", "SubjectID", "LibraryID", "SampleID", "StudyID", "StudyType", "Pipeline", "DiseaseCode", "SNOMED") \
    .where("SampleID = 'NA12878'") \
    .show()

In [0]:
meta_df \
    .dropna() \
    .cube("StudyID") \
    .count() \
    .show()

In [0]:
# Page Break

# Write to Deltatable

In [0]:
metadata_table = "./lakehouse/bcbio/metadata_table"

In [0]:
meta_df.write.format("delta").mode("overwrite").save(metadata_table)

In [0]:
!ls ./lakehouse/bcbio/metadata_table

In [0]:
# Page Break

# Summary

* In real world scenario, this could involve much more complex structure and/or integration with upstream systems.
    * For example, interfacing with systems such as RedCAP, FHIR, or Pathling, ontology and some kind of clinical Phenotype look up server
    * And there may be possibility with dynamically looking them up from these systems
* Consider: we need to ingest some minimal metadata info and, need to ETL out these minimal Phenotype info from these systems
* We may or may not need to ingest these meta information into data warehouse -- which depends on case by case basis.

In [0]:
# Page Break

# Stop Spark Session

In [0]:
spark.stop()

In [0]:
# Continue to next notebook